# Template Statistics Analysis
**Author**: Majid Vafaeezadeh  
**Date**: 2025-04-08

This notebook evaluates non-zero statistics and visualizes template masks gecls
nerated by the deformable registration pipeline.

In [ ]:
## 1. Load Libraries and Helper Functions
import numpy as np
import os
import sys

# Add the evaluations folder to the Python path
sys.path.append("../evaluations")
print("Added evaluations/ to Python path.")

# Import your helper functions
from eval_nonzero_stats import get_template_stats
from eval_visualize import show_template
print("Successfully imported helper functions!")

In [ ]:
## 2. Load and Evaluate Template Files
template_dir = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\templates\Template10allsimNonrigid"
#P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\output\1simNonrigid
#P:\Projects\DeepFlow\deepFlowDocker\output\Alignment\Template10allsimNonrigid
#P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\output\Template10allsimNonrigid
for file in sorted(os.listdir(template_dir)):
    if file.endswith(".npy"):
        filepath = os.path.join(template_dir, file)
        arr = np.load(filepath)

        stats = get_template_stats(arr)
        print(f"{file} → Stats: {stats}")

        show_template(arr, title=file, threshold=0.5)


In [ ]:
# This block of code compares the .npy template files from two directories
# and checks if their content is identical. For each pair of files,
# it loads them and compares the arrays to see if they are exactly the same.
# If they are identical, it will print a message stating so.

# Directories containing .npy files for comparison
template_dir_1 = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\output\1simNonrigid\old3"
template_dir_2 = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\output\1simNonrigid"

# Get all .npy files in both directories
files_1 = sorted([f for f in os.listdir(template_dir_1) if f.endswith(".npy")])
files_2 = sorted([f for f in os.listdir(template_dir_2) if f.endswith(".npy")])

# Ensure both directories have the same number of files
if len(files_1) != len(files_2):
    print("Warning: The number of template files in the two directories doesn't match.")

# Compare corresponding files from both directories
for file1, file2 in zip(files_1, files_2):
    file1_path = os.path.join(template_dir_1, file1)
    file2_path = os.path.join(template_dir_2, file2)

    # Load the .npy files
    arr1 = np.load(file1_path)
    arr2 = np.load(file2_path)

    # Check if the arrays are identical
    if np.array_equal(arr1, arr2):
        print(f"{file1} and {file2} are identical.")
    else:
        print(f"{file1} and {file2} are different.")


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass

template_dir = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\templates\Template10allsimNonrigid"
template_prefix = "template_phase"
num_templates = 30

areas = []
centers = []

plt.figure(figsize=(20, 6))
for i in range(num_templates):
    path = os.path.join(template_dir, f"{template_prefix}_{i:02d}_similarity_10.npy")
    if not os.path.exists(path):
        print(f"Missing: {path}")
        continue
    mask = np.load(path)
    area = np.sum(mask)
    com = center_of_mass(mask)
    
    areas.append(area)
    centers.append(com)
    
    # Plot each template
    plt.subplot(3, 10, i + 1)
    plt.imshow(mask, cmap='gray')
    plt.title(f"{i:02d}\nA={int(area)}")
    plt.axis('off')

plt.suptitle("Template Masks (30 phases) - Area & Shape", fontsize=16)
plt.tight_layout()
plt.show()

# Plot area vs phase
plt.figure()
plt.plot(range(num_templates), areas, marker='o')
plt.title("Template Area per Phase")
plt.xlabel("Frame Index")
plt.ylabel("Area (pixels)")
plt.grid(True)
plt.show()
